# LCEL 인터페이스
사용자 정의 체인을 가능한 쉽게 만들 수 있도록, Runnable 프로토콜을 구현
- stream: 응답의 청크를 스트리밍
- invoke: 입력에 대해 체인을 호출
- batch: 입력 목록에 대해 체인을 호출

비동기 메소드
- astream: 비동기적으로 응답의 청크를 스트리밍
- ainvoke: 비동기적으로 입력에 대해 체인을 호출
- abatch: 비동기적으로 입력 목록에 대해 체인 호출
- astream_log: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

model = ChatOllama(model = 'ollama-ko-0502')
prompt = PromptTemplate.from_template("{topic}에 대하여 3문장으로 설명해줘.")
chain = prompt | model | StrOutputParser()

In [2]:
input ={'topic': '멀티모달'}
chain.invoke(input)

'멀티모달이란, 사용자와 기계가 서로 다른 형태의 정보를 주고받는 것을 말합니다. 예를 들어, 사용자가 음성으로 질문을 하고, 기계가 텍스트로 대답하는 것이 멀티모달입니다'

## stream: 실시간 출력
chain.stream 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(content)을 즉시 출력.
end = "" 인자는 출력 후 줄바꿈을 하지 않도록 설정, flush=True 인자는 출력 버퍼를 즉시 비우도록 함.

In [4]:
answer = chain.stream(input)

In [5]:
answer

<generator object RunnableSequence.stream at 0x10deb5e40>

## batch:배치(단위 실행)
함수 chain.batch는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕서너리에 있는 topic키의 값을 사용하여 일괄 처리 수행
<br>max_concurrency 매개변수를 사용하여 동시 요청 수를 설정할 수 있다.<br>
config 딕셔너리는 max_concurrency 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정. 여기서는 최대 3개의 작엄을 동시에 처리

In [4]:
answer = chain.batch(
    [
        {"topic": "ChatCPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config = {"max_concurrnecy": 3},
)

In [8]:
print(answer)
print(f"\nIndex 0: {answer[0]}")

['ChatCPT는 인공지능 챗봇입니다. 사용자와 대화를 나누고, 질문에 대한 답변을 제공합니다. 모든 대답은 한국어로 대답해줍니다', '인스타그램은 사진과 동영상을 공유하는 소셜 미디어입니다. 인스타그램은 2010년 10월에 출시되었으며, 현재 전 세계적으로 5억 명 이상의 사용자를 보유하고 있습니다. 인스타그램은 페이스북이 운영합니다', '멀티모달이란, 사용자와 기계가 서로 다른 형태의 정보를 주고받는 것을 말합니다. 예를 들어, 사용자가 음성으로 질문을 하고, 기계가 텍스트로 대답하는 것이 멀티모달입니다', '인간이 컴퓨터를 이용하여 특정한 작업을 수행할 수 있도록 명령어를 작성하는 과정입니다. 프로그래머는 이러한 과정을 통해 프로그램을 만듭니다', '머신러닝이란 컴퓨터가 스스로 학습하는 기술입니다. 머신러닝은 데이터를 분석하고, 그 결과를 바탕으로 새로운 정보를 예측하거나 분류할 수 있습니다. 머신러닝을 통해 인공지능(AI) 로봇이나 자율주행 자동차 등이 만들어졌습니다']

Index 0: ChatCPT는 인공지능 챗봇입니다. 사용자와 대화를 나누고, 질문에 대한 답변을 제공합니다. 모든 대답은 한국어로 대답해줍니다


## async stream: 비동기 스트림

함수 chain.astream은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리<br>
비동기 for 루프(async for)를 사용하여 스트림에서 메세지를 순차적으로 받아오고, print함수를 통해 메시지의 내용(s.content)을 즉시 출력 <br> end=""는 출력 후 줄바꿈을 하지 않도록 설정하며, flush=True는 출력 버퍼를 강제로 비워 즉시 출력되도록 함.

In [9]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리
async for token in chain.astream({"topic":"YouTube"}):
    # 메시지 내용 출력, 줄바꿈 없이 바로 출력하고 버퍼 비우기
    print(token, end = "", flush=True)

YouTube는 동영상 공유 사이트입니다. 전 세계적으로 가장 인기 있는 비디오 공유 플랫폼 중 하나이며, 수십억 명의 사용자들이 매일 업로드된 동영상을 시청합니다. YouTube는 또한 음악 스트리밍 서비스를 제공하고 있습니다

# async invoke: 비동기 호출
chain 객체의 ainvoke 메서드는 비동기적으로 주어진 인자를 사용하여 작업 수행.<br>
여기서는 topic이라는 키와 NBDA(엔비디아의 티커)라는 값을 가진 딕셔너리를 인자로 전달.<br>
특정 토픽에 대한 처리를 비동기적으로 요청하는데 사용될 수 있다.

In [10]:
chain.invoke({"topic": "NVDA"})

'NVDA는 시각 장애인들을 위한 무료 소프트웨어입니다. NVDA는 컴퓨터를 사용하는 데 필요한 모든 기능을 제공합니다. NVDA는 음성 출력, 스크린 리더, 키보드 보조기능 등을 포함하고 있습니다'

In [11]:
my_process = chain.ainvoke({"topic":"NVDA"})

In [12]:
await my_process

'NVDA는 시각 장애인들을 위한 무료 소프트웨어입니다. NVDA는 컴퓨터를 사용하는 데 필요한 모든 기능을 제공합니다. NVDA는 음성 출력, 스크린 리더, 키보드 보조기능 등을 포함하고 있습니다'

## Parallel: 병렬성
LangChain Expression Language가 병렬 요청을 지원하는 방법.<br>
langchain_core.runnalbles 모듈의 RunnableParallel 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시<br>
ChatPromptTemplate.from_template 메서드를 사용하여 주어진 country에 대한 수도와 면적을 구하는 두개의 체인(chain1, chain2) 생성.<br>
capital과 area라는 키로 결합하여 동시에 실행할 수 있는 combined 객체 생성

In [14]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야??")
    | model
    | StrOutputParser()
)


combined = RunnableParallel(capital=chain1, area=chain2)


In [15]:
combined

{
  capital: PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디야?')
           | ChatOllama(model='ollama-ko-0502')
           | StrOutputParser(),
  area: PromptTemplate(input_variables=['country'], template='{country}의 면적은 얼마야??')
        | ChatOllama(model='ollama-ko-0502')
        | StrOutputParser()
}

In [16]:
chain1.invoke({"country":"대한민국"})

'서울입니다. 서울은 대한민국의 수도이고, 600년 이상 된 도시로써 세계에서 가장 오래된 수도 중 하나입니다. 서울은 한반도의 중심에 위치해 있으며, 북쪽으로는 강원도와 경기도, 남쪽으로는 충청남도와 경상북도, 서쪽으로는 전라북도와 황해도, 동쪽으로는 경상남도와 동해로 둘러싸여 있습니다'

In [17]:
chain2.invoke({"country":"대한민국"})

'한국의 전체 면적은 10만 3170km2입니다. 이는 세계에서 108번째로 큰 나라에요. 또한, 한국은 동쪽으로는 일본과 서쪽으로는 중국과 맞닿아 있어요'

In [18]:
combined.invoke({"country": "대한민국"})

{'capital': '서울입니다. 서울은 대한민국의 수도이고, 600년 이상 된 도시로써 세계에서 가장 오래된 수도 중 하나입니다. 서울은 한반도의 중심에 위치해 있으며, 북쪽으로는 강원도와 경기도, 남쪽으로는 충청남도와 경상북도, 서쪽으로는 전라북도와 황해도, 동쪽으로는 경상남도와 동해로 둘러싸여 있습니다',
 'area': '한국의 전체 면적은 10만 3170km2입니다. 이는 세계에서 108번째로 큰 나라에요. 또한, 한국은 동쪽으로는 일본과 서쪽으로는 중국과 맞닿아 있어요'}

In [21]:
result = combined.batch([{"country":"대한민국"},{"country":"미국"}])
print(result)
print(f"\nIndex 0:{result[0]}")

[{'capital': '서울입니다. 서울은 대한민국의 수도이고, 600년 이상 된 도시로써 세계에서 가장 오래된 수도 중 하나입니다. 서울은 한반도의 중심에 위치해 있으며, 북쪽으로는 강원도와 경기도, 남쪽으로는 충청남도와 경상북도, 서쪽으로는 전라북도와 황해도, 동쪽으로는 경상남도와 동해로 둘러싸여 있습니다', 'area': '한국의 전체 면적은 10만 3170km2입니다. 이는 세계에서 108번째로 큰 나라에요. 또한, 한국은 동쪽으로는 일본과 서쪽으로는 중국과 맞닿아 있어요'}, {'capital': '워싱턴 D.C.입니다. 미국의 수도인 워싱턴 D.C.는 1790년에 건설되었고, 현재 약 70만 명이 거주하고 있습니다', 'area': '미국은 9,629,091 평방 킬로미터입니다. 미국의 면적은 세계에서 가장 큰 나라 중 하나입니다. 미국은 아메리카 대륙의 대부분을 차지하고 있습니다'}]

Index 0:{'capital': '서울입니다. 서울은 대한민국의 수도이고, 600년 이상 된 도시로써 세계에서 가장 오래된 수도 중 하나입니다. 서울은 한반도의 중심에 위치해 있으며, 북쪽으로는 강원도와 경기도, 남쪽으로는 충청남도와 경상북도, 서쪽으로는 전라북도와 황해도, 동쪽으로는 경상남도와 동해로 둘러싸여 있습니다', 'area': '한국의 전체 면적은 10만 3170km2입니다. 이는 세계에서 108번째로 큰 나라에요. 또한, 한국은 동쪽으로는 일본과 서쪽으로는 중국과 맞닿아 있어요'}
